<p>
<font size='5' face='Georgia, Arial'>IIC-2233 Apunte Programación Avanzada</font><br>
<font size='1'>&copy; 2015 Karim Pichara - Christian Pieringer. Todos los derechos reservados.</font>
<br>
<font size='1'> Modificado en 2017-2, 2018-1, 2018-2, 2019-1 y 2019-2 por Equipo Docente IIC2233</font>
</p>

## Clases de Excepciones

En Python, todas las excepciones heredan de `BaseException`. A partir de ella existen tres tipos de excepciones: **`SystemExit`**, **`KeyboardInterrupt`**, y **`Exception`**. Todas las excepciones generadas por errores durante la ejecución de un programa son subclases de `Exception`, tal como se muestra en el siguiente diagrama:

![](img/jerarquia-excepciones.png)

Esto quiere decir, que si se usa `Exception` para manejar los errores, actuaremos sobre todas las subclases de `Exception`. De esta forma el tratamiento es general y no específico a un tipo de error en especial. En general es recomendable actuar de forma selectiva sobre un tipo determinado de excepciones (`IOError`, `AtributeError`, `ValueError`, etc.), sin embargo, existen otros casos en que no se sabe por cuál razón el programa podría fallar en los que conviene actuar de manera general usando `Exception`.

In [1]:
def dividir(num,den):
    if not (isinstance(num, int) and isinstance(den, int)):
        raise TypeError("Error de tipo en numerador o denominador. :'(")

    if num < 0 or den < 0:
        raise ValueError("Hay un valor negativo entre numerador y denominador >:(")

    return float(num)/float(den)

In [2]:
try:
    print(dividir(4,0))
    
except Exception as err:
    # Este bloque opera para todos los tipos de excepciones que hereden de Exception
    print(f"Error: {err}")
    print("Revise los datos de entrada. Algo pasó ahí, pero no sé qué tipo específico de Exception ... ")

Error: float division by zero
Revise los datos de entrada. Algo pasó ahí, pero no sé qué tipo específico de Exception ... 


## Excepciones personalizadas

Podemos crear nuestros propios tipos de excepciones. Para ello debemos heredar desde la clase `Exception`. Podemos modificar el comportamiento heredado sobreescribiendo los métodos que tiene implementada esta clase.

In [3]:
class Excepcion1(Exception):
    # Al no sobreescribir nada, hereda todo sin modificaciones
    pass


class Excepcion2(Exception):
    def __init__(self, a, b):
        # Sobreescribimos el __init__ para cambiar el ingreso de los parámetros
        super().__init__(f"Alguno de los valores {a} o {b} no es entero\n")


def dividir(num, den):
    # Por ejemplo, redefiniremos las excepciones que
    # utilizamos en los ejemplos anteriores.
    if not (isinstance(num, int) and isinstance(den, int)):
        raise Excepcion2(num, den)

    if num < 0 or den < 0:
        raise Excepcion1("Los valores son negativos\n")

    return float(num) / float(den)

In [4]:
# Este ejempo lanza una Excepcion1
try:
    print(dividir(4, -3))

except Excepcion1 as err:
    # Este bloque opera para la Excepcion1
    print(f"Error: {err}")

except Excepcion2 as err:
    # Este bloque opera para Excepcion2 cuando los datos no son enteros
    print(f"Error: {err}")

Error: Los valores son negativos



In [5]:
# Este ejemplo lanza una Excepcion2
try:
    print(dividir(4.4, -3))

except Excepcion1 as err:
    # Este bloque opera para la Excepcion1
    print(f"Error: {err}")

except Excepcion2 as err:
    # Este bloque opera para Excepcion2 cuando los datos no son enteros
    print(f"Error: {err}")


Error: Alguno de los valores 4.4 o -3 no es entero



Podemos definir comportamientos personalizados para las excepciones que creamos como, por ejemplo, agregar métodos que nos permitan recuperar información de la excepción.

In [6]:
class ErrorTransaccion(Exception):
    
    def __init__(self, fondos, gasto):
        super().__init__(f"El dinero en la billetera no alcanza para pagar ${gasto}")
        self.fondos = fondos
        self.gasto = gasto
    
    def exceso(self):
        return self.gasto - self.fondos


class Billetera:
    
    def __init__(self, dinero):
        self.fondos = dinero
    
    def pagar(self, gasto):
        if self.fondos - gasto < 0:
            raise ErrorTransaccion(self.fondos, gasto)
        self.fondos -= gasto

        
b = Billetera(1000)

try:
    b.pagar(1500)

except ErrorTransaccion as err:
    print(f"Error: {err}. Hay un exceso de gastos de ${err.exceso()}.")

Error: El dinero en la billetera no alcanza para pagar $1500. Hay un exceso de gastos de $500.


## Un ejemplo: La lista del curso

Este ejemplo desea modelar la siguiente situación: se desean inscribir alumnes en distintos cursos universitarios, para lo cual se cuentan con listas de las peticiones de varios estudiantes por curso. El problema, es que estas listas fueron afectadas por *hackers* que poblaron de información falsa, específicamente, hay alumnes con número de identificación duplicados. Ya que no se tiene como identificar los falsos de los verdaderos, se incribirán solamente aquelles con un identificador nuevo, y los duplicados de almacenará de alguna forma, para futura referencia.

La siguiente solución utiliza una excepción personalizada para llevar el registro de los duplicados encontrados en la lista. Aprovecha el uso de atributos de clase para mantener un registro general de todos los duplicados, y atributos de instancia para obtener información.

In [7]:
from collections import namedtuple, defaultdict

# Modelamos les alumnes como entidades simples
Alumne = namedtuple('Alumne', ['numero', 'nombre', 'apellido'])

class ErrorAlumneRepetide(Exception):
    
    # Se mantiene un diccionario como atributo de clase para almacenar todos los duplicados
    alumnes_repetides = defaultdict(list)
    
    def __init__(self, alumne):
        # Se almacena le alumne que lanzó la excepción
        self.alumne = alumne
        numero = alumne.numero
        # Se almacena el duplicado
        self.alumnes_repetides[numero].append(alumne)
        super().__init__('¡Error de alumne repetide de número: {numero}')
    
    # Creamos property de instancia que accede a todos los duplicados del mismo número
    @property
    def repetides(self):
        return self.alumnes_repetides[self.alumne.numero]
    
    # Creamos property de instancia que calcula la cantidad de duplicados del mismo número
    @property
    def cantidad(self):
        return len(self.alumnes_repetides[self.alumne.numero])
    
    
# Modelamos el curso con una estructura simple para almacenar sus alumnes
class Curso:
    
    def __init__(self, nombre):
        self.nombre = nombre
        self.alumnes = dict()
    
    # Método encargado de inscribir nuevo alumne, en caso de encontrar duplicado, lanza error
    def inscribir(self, alumne):
        if alumne.numero in self.alumnes:
            raise ErrorAlumneRepetide(alumne)
        else:
            self.alumnes[alumne.numero] = alumne
            print(f'✅ Alumne de número {alumne.numero} inscrito exitosamente.\n')


In [8]:
alumnes_no_inscritos = [
    Alumne('15633459', 'Juan', 'Hernández'),
    Alumne('1663525J', 'Belén', 'Pinto'),
    Alumne('17632451', 'Ariel', 'Gonzalez'),
    Alumne('15633459', 'Fernanda', 'Errazuriz'),
    Alumne('1563001J', 'Javiera', 'Martínez'),
    Alumne('1663525J', 'Benjamín', 'Valdivieso'),
    Alumne('15633459', 'Cristian', 'Soto'),
]

iic2233 = Curso('IIC2233')

for alumne in alumnes_no_inscritos:
    try:
        iic2233.inscribir(alumne)
    except ErrorAlumneRepetide as error:
        print(f'❌ Alumne repetido de número {alumne.numero} encontrado')
        print(f'Ya van {error.cantidad} repetidos con el mismo número:')
        print(', '.join([f'{alumne.nombre} {alumne.apellido}' for alumne in error.repetides]))
        print()

✅ Alumne de número 15633459 inscrito exitosamente.

✅ Alumne de número 1663525J inscrito exitosamente.

✅ Alumne de número 17632451 inscrito exitosamente.

❌ Alumne repetido de número 15633459 encontrado
Ya van 1 repetidos con el mismo número:
Fernanda Errazuriz

✅ Alumne de número 1563001J inscrito exitosamente.

❌ Alumne repetido de número 1663525J encontrado
Ya van 1 repetidos con el mismo número:
Benjamín Valdivieso

❌ Alumne repetido de número 15633459 encontrado
Ya van 2 repetidos con el mismo número:
Fernanda Errazuriz, Cristian Soto

